# 1. IMPORT LIBRARIES AND DATASET

In [ ]:
%pip install pandas
%pip install openpyxl
%pip install matplotlib

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 325.1 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.8 kB 372.4 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 404.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.2/11.6 MB 3.9 MB/s eta 0:00:03
   - -------------------------------------- 0.4/11.6 MB 3.9 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/11.6 MB 7.3 MB/s eta 0:00:02
   ------- -------------------------------- 2.3/11.6 MB 13.0 MB/s eta 0:00:01
   ----------- ---------------------------- 3.3/11.6 MB 15.2 MB/s eta 0:00:01
   ------------------- -------------------- 5.7/11.6 MB 21.4 MB/s eta 0:00:01
   --------------------------- ------------ 8.1/11.6 MB 25.7 MB/s eta 0:00:01
  


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/108.2 kB ? eta -:--:--
     --------------------------------- ----- 92.2/108.2 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 108.2/108.2 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.2/8.1 MB 7.4 MB/s eta 0:00:02
   -- ------------------------------------- 0.5/8.1 MB 6.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/8.1 MB 9.2 MB/s eta 0:00:01
   -------------- ------------------------- 2.9/8.1 MB 16.8 MB/s eta 0:00:01
   ----------------------- ---------------- 4.6/8.1 MB 21.2 MB/s eta 0:00:01
   ---------------------------------- ----- 7.0/8.1 MB 26.4 MB/s eta 0:00:01
   ---------------------------------------  8.1/8.1 MB 28.5 MB/s eta 0:00:01
   ---------------------------------------  8.1/8.1 MB 28.5 MB/s eta 0:00:01
   ---------------------------------------  8.1/8.1 MB 28.5 MB/s eta 0:00:01
   -----


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import string
import re

Matplotlib is building the font cache; this may take a moment.


In [ ]:


df = pd.read_excel('../data/careerviet.xlsx')
print(df.info())
print(df.describe())
print(df.isnull().sum())


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 435.7 kB/s eta 0:00:01
   ------- ------------------------------- 51.2/250.9 kB 375.8 kB/s eta 0:00:01
   ------------------------ ------------- 163.8/250.9 kB 984.6 kB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150 entries, 0 to 1149
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   image href         1132 non-null   object
 1   lazy-img src       1150 non-null   object
 2   job_link           1150 non-null   object
 3   job_link href      1150 non-null   object
 4   compan

# PREPROCESS - TÁCH CỘT

## CỘT JOBDETAIL
Cần lấy các thông tin về: Lương và yêu cầu năm kinh nghiệm

In [ ]:
print(df.head())

              salary  min_salary  max_salary
0     trên 18 tr vnd  18000000.0         NaN
1    2 tr - 3 tr vnd         NaN         NaN
2  1,000 - 1,500 usd  25485000.0  38227500.0


In [5]:
# Hàm xử lý từng dòng trong cột 'jobdetail'
def process_text(text):
    # Loại bỏ '\n' và khoảng trắng thừa
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Lấy đoạn giữa "Mức lương" và "Địa điểm", nếu không thấy địa điểm thì thay bằng "Số lượng tuyển"
    salary_info = text[text.find('Mức lương') + len('Mức lương:'):text.find('Địa điểm')].strip()
    if salary_info == '':
        salary_info = text[text.find('Mức lương') + len('Mức lương:'):text.find('Số lượng tuyển')].strip()
    # Nếu salary_info vẫn rỗng thì tìm "Thỏa thuận", nếu thấy thì lưu thành thỏa thuận
    if salary_info == '':
        if 'Thỏa thuận' in text:
            salary_info = 'Thỏa thuận'
    # Lấy đoạn giữa "Kinh nghiệm" và "Xem số người đã ứng tuyển"
    experience_info = text[text.find('Kinh nghiệm') + len('Kinh nghiệm:'):text.find('Xem số người đã ứng tuyển')].strip()
    
    return salary_info, experience_info

# Áp dụng hàm cho toàn bộ cột 'jobdetail'
df['raw_salary'], df['raw_experience'] = zip(*df['jobdetail'].apply(process_text))

# Kết quả
print(df)

                                             Unnamed: 0  \
0     https://www.topcv.vn/viec-lam/brse-ky-su-cau-n...   
1     https://www.topcv.vn/viec-lam/cobol-developer/...   
2     https://www.topcv.vn/viec-lam/frontend-teamlea...   
3     https://www.topcv.vn/viec-lam/fullstack-develo...   
4     https://www.topcv.vn/viec-lam/intern-nodejs-de...   
...                                                 ...   
4495  https://www.topcv.vn/viec-lam/chuyen-vien-phat...   
4496  https://www.topcv.vn/viec-lam/thuc-tap-sinh-ai...   
4497  https://www.topcv.vn/viec-lam/mobile-developer...   
4498  https://www.topcv.vn/viec-lam/thuc-tap-sinh-xa...   
4499  https://www.topcv.vn/viec-lam/ky-su-cau-noi-br...   

                                               jobtitle  \
0     BrSE\n        -\n            Kỹ Sư Cầu Nối\n  ...   
1                          Cobol\n            Developer   
2                            Frontend\n        Teamlead   
3                 Fullstack Developer\n        (Senior)

In [6]:
#print df head of 2 new columns
print(df[['raw_salary', 'raw_experience']].head())


   raw_salary             raw_experience
0  Thoả thuận                      1 năm
1  Thoả thuận                      3 năm
2  Thoả thuận                      3 năm
3  Thoả thuận                      2 năm
4  Thoả thuận  Không yêu cầu kinh nghiệm


## CỘT JOBNAME
Xóa "Chuyên môn" và khoảng trắng thừa

In [32]:
# đọc file vào DataFrame df
df = pd.read_excel('../data/careerviet.xlsx')

# Tạo cột mới job_title từ job_link gốc
df['job_title'] = df['job_link']

In [46]:
# Tạo lại cột job_title từ job_link nếu chưa có
if 'job_title' not in df.columns:
    df['job_title'] = df['job_link']

# Chuẩn hóa tên công việc
def normalize_title(title):
    if pd.isna(title):
        return None
    # Bỏ khoảng trắng đầu cuối và viết thường
    title = title.strip().lower()
    # Xóa dấu ngoặc vuông và tròn, nếu có
    title = re.sub(r"[\[\]()]", "", title)
    # Bỏ khoảng trắng thừa
    title = re.sub(r"\s+", " ", title)
    return title

# Áp dụng nếu muốn chuẩn hóa
df['job_title_clean'] = df['job_title'].apply(normalize_title)

# Hiển thị kết quả
df[['job_link', 'job_title', 'job_title_clean']]

,job_link,job_title,job_title_clean
0,Documentation Staff (Chemicals Department) - T...,Documentation Staff (Chemicals Department) - T...,documentation staff chemicals department - tra...
1,Senior Devops,Senior Devops,senior devops
2,Product Manager,Product Manager,product manager
3,Sales Engineer,Sales Engineer,sales engineer
4,AI Engineer (AI Agent & Automation),AI Engineer (AI Agent & Automation),ai engineer ai agent & automation
...,...,...,...
1145,Junior Tester,Junior Tester,junior tester
1146,Kỹ Thuật Viên Viễn Thông,Kỹ Thuật Viên Viễn Thông,kỹ thuật viên viễn thông
1147,Sales Manager – IT Background,Sales Manager – IT Background,sales manager – it background
1148,IT,IT,it


## CỘT COMPANY SCALE

In [8]:
# print company scale unique
print(df['companyscale'].unique())

['nhân viên' '100-499 nhân viên' '10-24 nhân viên' '25-99 nhân viên'
 '500-1000 nhân viên' '1000+ nhân viên' '10000+ nhân viên'
 '5000+ nhân viên' '3000+ nhân viên' '1-9 nhân viên' '100-499' '25-99'
 '500-1000' '10000+' '5000+' '1000+']


In [9]:
# Xóa 'nhân viên' nếu xuất hiện, xóa khoảng trắng thừa
df['companyscale'] = df['companyscale'].apply(lambda x: re.sub(r'nhân viên', '', x).strip())

In [10]:
print(df['companyscale'].unique())

['' '100-499' '10-24' '25-99' '500-1000' '1000+' '10000+' '5000+' '3000+'
 '1-9']


In [11]:
#rename: from 1000+ to 1000 - 2999, from 3000+ to 3000 - 4999, from 500+0 to 5000 - 9999
df['companyscale'] = df['companyscale'].apply(lambda x: re.sub(r'1000\+', '1000 - 2999', x))
df['companyscale'] = df['companyscale'].apply(lambda x: re.sub(r'3000\+', '3000 - 4999', x))
df['companyscale'] = df['companyscale'].apply(lambda x: re.sub(r'5000\+', '5000 - 9999', x))

print(df['companyscale'].unique())

['' '100-499' '10-24' '25-99' '500-1000' '1000 - 2999' '10000+'
 '5000 - 9999' '3000 - 4999' '1-9']


## COMPANY NAME


In [34]:
df = pd.read_excel('../data/careerviet.xlsx')

In [47]:
# Hàm chuẩn hóa tên công ty
def clean_company_name(name):
    if pd.isna(name):
        return None
    
    # Bỏ khoảng trắng thừa, dấu chấm cuối
    name = name.strip().rstrip('.')
    
    # Chuyển về chữ thường để xử lý
    name = name.lower()

    # Loại bỏ các từ khóa không cần thiết
    name = re.sub(
        r'\b(công ty|cty|c\.?t\.?y\.?|công ty cổ phần|công ty tnhh|tnhh|cp|cổ phần|trách nhiệm hữu hạn|ctcp|co\.?|ltd|inc|jsc|gmbh|llc|s\.?a\.?)\b',
        '',
        name
    )

    # Loại bỏ các ký tự không cần thiết
    name = re.sub(r'[\.\(\)\-]', '', name)

    # Loại bỏ khoảng trắng dư thừa
    name = re.sub(r'\s+', ' ', name).strip()

    # Viết hoa đầu mỗi từ
    name = string.capwords(name)

    return name

# Áp dụng vào DataFrame
df['company_clean_name'] = df['company-name'].apply(clean_company_name)

# Xem kết quả
print(df[['company-name', 'company_clean_name']])

                                 company-name  \
0                                     Bảo Mật   
1     Ngân Hàng Tmcp Sài Gòn - Hà Nội ( Shb )   
2                                  Careerviet   
3                      Công Ty Cổ Phần Netcab   
4                             Wanek Furniture   
...                                       ...   
1145                             Employertest   
1146                   Công Ty Cổ Phần Kasati   
1147                Công Ty General Resources   
1148           Công Ty Tnhh Kenstone Việt Nam   
1149               Công Ty Tnhh Acer Việt Nam   

                     company_clean_name  
0                               Bảo Mật  
1     Ngân Hàng Tmcp Sài Gòn Hà Nội Shb  
2                            Careerviet  
3                                Netcab  
4                       Wanek Furniture  
...                                 ...  
1145                       Employertest  
1146                             Kasati  
1147                  General Res

## 4 CỘT SKILL_OFFER

In [13]:
# Fill NaN values with an empty string
df['skill_offer1'] = df['skill_offer1'].fillna('')

# Xóa dấu chấm cuối câu, xóa khoảng trống thừa, chuyển về chữ thường skill_offer1
df['skill_offer1'] = df['skill_offer1'].apply(lambda x: x.strip().lower())

# viết hoa đầu mỗi từ
df['skill_offer1'] = df['skill_offer1'].apply(lambda x: string.capwords(x))

# print unique
print(df['skill_offer1'].unique())

[''
 '- Thành Thạo Vi Tính (word, Excel), Canva Cơ Bản - Hiểu Nhu Cầu Tìm Kiếm Của Người Dùng: Biết Được Người Đọc Cần Gì Khi Lên Google Tìm Kiếm Thông Tin Để Triển Khai, Sắp Xếp Thông Tin Phù Hợp. - Có Hiểu Biết Cơ Bản Về Seo: Seo Là Gì? Vai Trò Của'
 '.net' '.net Framework' '.net Framework .net Core .net' '2d Graphics'
 '3d Artist' '3d Max' 'Abap Development' 'Actionscript' 'Active Listening'
 'Adobe Illustrator' 'Adobe Photoshop' 'Advanced English' 'After Effect'
 'Agavi' 'Agile' 'Ai' 'An Ninh Mạng' 'An Ninh Mạng (cybersecurity)'
 'Analytics' 'Android' 'Android Sdk' 'Angular' 'Angular 2' 'Angularjs'
 'Animation' 'Apex' 'Apex Development' 'Api' 'Api Restful' 'Api Testing'
 'Api Testing (rest, Soap)' 'Asp.net' 'Attention To Detail' 'Autocad'
 'Automation Test' 'Aws'
 'Aws Services (e.g., S3, Ec2, Rds, Redshift, Emr, Glue, Lambda)'
 'Azure Devops' 'Ba' 'Backend' 'Backend Developer'
 'Back-end Web Development' 'Bán Hàng' 'Banking' 'Báo Cáo Lỗi'
 'Bảo Mật Thông Tin' 'Bình Thạnh' 'Blockch

In [14]:
# Fill NaN values with an empty string
df['skill_offer2'] = df['skill_offer2'].fillna('')

# Xóa dấu chấm cuối câu, xóa khoảng trống thừa, chuyển về chữ thường skill_offer
df['skill_offer2'] = df['skill_offer2'].apply(lambda x: x.strip().lower())

# viết hoa đầu mỗi từ
df['skill_offer2'] = df['skill_offer2'].apply(lambda x: string.capwords(x))

# print unique
print(df['skill_offer2'].unique())

['' 'Làm Việc Độc Lập' 'Asp.net' '3d Artist' 'Javascript' 'C#' 'Android'
 'Java' 'Jquery' 'C++' 'Python'
 'Kiến Thức Về Các Hệ Thống Quản Trị Cơ Sở Dữ Liệu (sql)' 'Angularjs'
 'Sql Server' 'Css' 'Unity' 'Actionscript' 'Fullstack Developer'
 'Asp.net Mvc Asp.net Core Mvc' 'Graphic Designer' 'Game Artist'
 'Character Animation' 'Moho/ After Effect' 'Photoshop' 'Adobe Photoshop'
 '2d Animation' 'Illustrator' 'Agavi' 'Illustration' '3d Max'
 'Drawing Skills' '3d Modeling (blender, 3ds Max, Cinema 4d, Maya, V.v.)'
 '3d Modeling' 'Texturing' 'Ielts' 'Problem-solving' 'Uv Unwrapping'
 '3d Modeling (blender, Maya, 3ds Max, Zbrush, Etc.)'
 'Version Control (git)' 'Leadership'
 '3d Modeling (maya, 3ds Max, Blender)' 'Maya'
 '3d Modeling Software (e.g., Blender, Maya, 3ds Max, Zbrush)'
 'Khả Năng Sáng Tạo Và Tư Duy Hình Ảnh' 'Autocad' 'Revit'
 'Sap Modules (at Least One, E.g., Sd, Mm, Fi, Co)' 'Adobe After Effects'
 'Adobe Illustrator' 'Thiết Kế Game' 'Iso 9001' 'Bridge Engineer' 'Scrum'
 'Projec

In [15]:
# Fill NaN values with an empty string
df['skill_offer3'] = df['skill_offer3'].fillna('')

# Xóa dấu chấm cuối câu, xóa khoảng trống thừa, chuyển về chữ thường skill_offer
df['skill_offer3'] = df['skill_offer3'].apply(lambda x: x.strip().lower())

# viết hoa đầu mỗi từ
df['skill_offer3'] = df['skill_offer3'].apply(lambda x: string.capwords(x))

# print unique
print(df['skill_offer3'].unique())

['' 'R&d' 'Database' ...
 'Hiểu Biết Sâu Rộng Về Blockchain Và Cryptocurrency'
 'Kiểm Thử Ứng Dụng Di Động' 'Học Sâu (deep Learning)']


In [16]:
# Fill NaN values with an empty string
df['skill_offer4'] = df['skill_offer4'].fillna('')

# Xóa dấu chấm cuối câu, xóa khoảng trống thừa, chuyển về chữ thường skill_offer
df['skill_offer4'] = df['skill_offer4'].apply(lambda x: x.strip().lower())

# viết hoa đầu mỗi từ
df['skill_offer4'] = df['skill_offer4'].apply(lambda x: string.capwords(x))

# print unique
print(df['skill_offer4'].unique())

['' 'Iot' 'Windows Server' ...
 'Phát Triển Ứng Dụng Di Động (android Ios Hoặc Cả Hai)'
 'Java Hoặc Kotlin (cho Android)' 'Xây Dựng Mô Hình Ai']


## CỘT GENERAL_INFO

In [17]:
# Hàm xử lý từng dòng trong cột 'general_info'
def process_general_info(text):
    # Loại bỏ '\n' và khoảng trắng thừa
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Lấy đoạn giữa "Cấp bậc" và "Số lượng tuyển"
    # Nếu không tìm thấy thì trả về chuỗi rỗng
    position = text[text.find('Cấp bậc') + len('Cấp bậc:'):text.find('Số lượng tuyển')].strip()

    # Lấy đoạn giữa "Hình thức làm việc" và "Giới tính"
    # Nếu không tìm Giới tính thì in ra chuỗi sau "Hình thức làm việc"
    work_type = text[text.find('Hình thức làm việc') + len('Hình thức làm việc:'):text.find('Giới tính')].strip()

    return position, work_type

In [18]:
text = df['general_info'][0]
print(process_general_info(text))

('Nhân viên', 'Toàn thời gian')


In [19]:
# Fill NaN values with an empty string
df['general_info'] = df['general_info'].fillna('')

# Áp dụng hàm cho toàn bộ cột 'general_info'
df['position'], df['work_type'] = zip(*df['general_info'].apply(process_general_info))

# Kết quả
print(df)

                                             Unnamed: 0  \
0     https://www.topcv.vn/viec-lam/brse-ky-su-cau-n...   
1     https://www.topcv.vn/viec-lam/cobol-developer/...   
2     https://www.topcv.vn/viec-lam/frontend-teamlea...   
3     https://www.topcv.vn/viec-lam/fullstack-develo...   
4     https://www.topcv.vn/viec-lam/intern-nodejs-de...   
...                                                 ...   
4495  https://www.topcv.vn/viec-lam/chuyen-vien-phat...   
4496  https://www.topcv.vn/viec-lam/thuc-tap-sinh-ai...   
4497  https://www.topcv.vn/viec-lam/mobile-developer...   
4498  https://www.topcv.vn/viec-lam/thuc-tap-sinh-xa...   
4499  https://www.topcv.vn/viec-lam/ky-su-cau-noi-br...   

                                               jobtitle  \
0     BrSE\n        -\n            Kỹ Sư Cầu Nối\n  ...   
1                          Cobol\n            Developer   
2                            Frontend\n        Teamlead   
3                 Fullstack Developer\n        (Senior)

In [20]:
#delete all in position after "Học vấn"
df['position'] = df['position'].apply(lambda x: x.split('Học vấn')[0])
# delete blank
df['position'] = df['position'].apply(lambda x: x.strip())

In [21]:
#if position = '', then fill with "Nhân viên"
df['position'] = df['position'].apply(lambda x: 'Nhân viên' if x == '' else x)

In [22]:
#print unique
print(df['position'].unique())

['Nhân viên' 'Trưởng nhóm' 'Thực tập sinh' 'Trưởng/Phó phòng' 'Giám đốc'
 'Phó giám đốc' 'Quản lý / Giám sát' 'Trưởng chi nhánh']


In [23]:
#delete all after "Cấp bậc"
df['work_type'] = df['work_type'].apply(lambda x: x.split('Cấp bậc')[0])
df['work_type'] = df['work_type'].apply(lambda x: x.split('Yêu cầu khác')[0])
# delete blank
df['work_type'] = df['work_type'].apply(lambda x: x.strip())
#toàn thời gia -> toàn thời gian
# toàn thời gian -> giữ nguyên
df['work_type'] = df['work_type'].apply(lambda x: 'Toàn thời gian' if x == 'Toàn thời gia' else x)
df['work_type'] = df['work_type'].apply(lambda x: 'Bán thời gian' if x == 'Bán thời gia' else x)
df['work_type'] = df['work_type'].apply(lambda x: 'Thực tập' if x == 'Thực tậ' else x)

In [24]:
print(df['work_type'].unique())

['Toàn thời gian' 'Bán thời gian' 'Thực tập' '']


## CỘT LOCATION

In [7]:
provinces = [
    "Hà Nội",
    "Vĩnh Phúc",
    "Bắc Ninh",
    "Quảng Ninh",
    "Hải Dương",
    "Hải Phòng",
    "Hưng Yên",
    "Thái Bình",
    "Hà Nam",
    "Nam Định",
    "Ninh Bình",
    "Hà Giang",
    "Cao Bằng",
    "Bắc Kạn",
    "Tuyên Quang",
    "Lào Cai",
    "Yên Bái",
    "Thái Nguyên",
    "Lạng Sơn",
    "Bắc Giang",
    "Phú Thọ",
    "Điện Biên",
    "Lai Châu",
    "Sơn La",
    "Hoà Bình",
    "Thanh Hoá",
    "Nghệ An",
    "Hà Tĩnh",
    "Quảng Bình",
    "Quảng Trị",
    "Thừa Thiên Huế",
    "Đà Nẵng",
    "Quảng Nam",
    "Quảng Ngãi",
    "Bình Định",
    "Phú Yên",
    "Khánh Hoà",
    "Ninh Thuận",
    "Bình Thuận",
    "Kon Tum",
    "Gia Lai",
    "Đắk Lắk",
    "Đắk Nông",
    "Lâm Đồng",
    "Bình Phước",
    "Tây Ninh",
    "Bình Dương",
    "Đồng Nai",
    "Bà Rịa - Vũng Tàu",
    "Hồ Chí Minh",
    "Long An",
    "Tiền Giang",
    "Bến Tre",
    "Trà Vinh",
    "Vĩnh Long",
    "Đồng Tháp",
    "An Giang",
    "Kiên Giang",
    "Cần Thơ",
    "Hậu Giang",
    "Sóc Trăng",
    "Bạc Liêu",
    "Cà Mau"
]

# In ra danh sách
for province in provinces:
    print(province)

Hà Nội
Vĩnh Phúc
Bắc Ninh
Quảng Ninh
Hải Dương
Hải Phòng
Hưng Yên
Thái Bình
Hà Nam
Nam Định
Ninh Bình
Hà Giang
Cao Bằng
Bắc Kạn
Tuyên Quang
Lào Cai
Yên Bái
Thái Nguyên
Lạng Sơn
Bắc Giang
Phú Thọ
Điện Biên
Lai Châu
Sơn La
Hoà Bình
Thanh Hoá
Nghệ An
Hà Tĩnh
Quảng Bình
Quảng Trị
Thừa Thiên Huế
Đà Nẵng
Quảng Nam
Quảng Ngãi
Bình Định
Phú Yên
Khánh Hoà
Ninh Thuận
Bình Thuận
Kon Tum
Gia Lai
Đắk Lắk
Đắk Nông
Lâm Đồng
Bình Phước
Tây Ninh
Bình Dương
Đồng Nai
Bà Rịa - Vũng Tàu
Hồ Chí Minh
Long An
Tiền Giang
Bến Tre
Trà Vinh
Vĩnh Long
Đồng Tháp
An Giang
Kiên Giang
Cần Thơ
Hậu Giang
Sóc Trăng
Bạc Liêu
Cà Mau


In [8]:
def process_location(text):
    # Loại bỏ '\n' và khoảng trắng thừa
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\s+', ' ', text)

    # Nếu có thì chỉ lấy duy nhất tỉnh thành đó và xóa những thông tin còn lại
    return text


In [9]:
# Fill NaN values with an empty string
df['location'] = df['location'].fillna('')

# áp dụng vào cột Location
df['location'] = df['location'].apply(process_location)

# Kết quả
print(df['location'])


0       Hồ Chí Minh
1            Hà Nội
2       Hồ Chí Minh
3       Hồ Chí Minh
4        Bình Dương
           ...     
1145    Hồ Chí Minh
1146    Hồ Chí Minh
1147    Hồ Chí Minh
1148       Đồng Nai
1149         Hà Nội
Name: location, Length: 1150, dtype: object


In [10]:
# Hàm tìm các tỉnh thành trong văn bản, các hàng có nhiều hơn 1 tỉnh, ngăn cách nhau bởi dấu ','
def find_provinces(text):
    # Tìm các tỉnh thành trong văn bản
    found_provinces = [province for province in provinces if province in text]
    
    # Nếu không tìm thấy thì trả về chuỗi rỗng
    if len(found_provinces) == 0:
        return ''
    
    # Nếu tìm thấy thì trả về chuỗi các tỉnh thành, ngăn cách nhau bởi dấu ','
    return ', '.join(found_provinces)

# Áp dụng hàm cho toàn bộ cột 'Location'
df['location'] = df['location'].apply(find_provinces)

# Kết quả
print(df['location'])

0       Hồ Chí Minh
1            Hà Nội
2       Hồ Chí Minh
3       Hồ Chí Minh
4        Bình Dương
           ...     
1145    Hồ Chí Minh
1146    Hồ Chí Minh
1147    Hồ Chí Minh
1148       Đồng Nai
1149         Hà Nội
Name: location, Length: 1150, dtype: object


In [11]:
# print unique
print(df['location'].unique())

['Hồ Chí Minh' 'Hà Nội' 'Bình Dương' '' 'Đồng Nai' 'Quảng Ninh'
 'Bà Rịa - Vũng Tàu' 'Quảng Ngãi' 'Hải Phòng' 'Long An' 'Đà Nẵng'
 'Bắc Ninh' 'Hà Nam' 'Hải Dương' 'Hưng Yên' 'Cần Thơ' 'Phú Thọ'
 'Thái Nguyên' 'Nghệ An' 'Tây Ninh' 'Tiền Giang' 'Hậu Giang' 'Vĩnh Phúc']


## COMPANY_FIELD

In [30]:
## Company field nếu thiếu thì print ra "Khác"
df['company_field'] = df['company_field'].fillna('Khác')

# Xóa dấu chấm cuối câu, xóa khoảng trống thừa, chuyển về chữ thường
df['company_field'] = df['company_field'].apply(lambda x: x.strip().lower())

# viết hoa đầu mỗi từ
df['company_field'] = df['company_field'].apply(lambda x: string.capwords(x))


# PREPROCESS - XỬ LÝ CỘT DỮ LIỆU ĐẶC BIỆT
- Lương
- Kinh nghiệm


## Salary

In [13]:
# print unique salary
print(df['salary'].unique())


['Lương: Cạnh tranh' 'Lương: Cạnh Tranh' 'Lương: 1,200 - 1,500 USD'
 'Lương: 22 Tr - 45 Tr VND' 'Lương: 10 Tr - 13 Tr VND'
 'Lương: 15 Tr - 25 Tr VND' 'Lương: 8 Tr - 16 Tr VND'
 'Lương: Trên 20 Tr VND' 'Lương: 8 Tr - 20 Tr VND'
 'Lương: 15 Tr - 50 Tr VND' 'Lương: 500 - 1,000 USD'
 'Lương: 10 Tr - 18 Tr VND' 'Lương: 12 Tr - 20 Tr VND'
 'Lương: 15 Tr - 20 Tr VND' 'Lương: 30 Tr - 35 Tr VND'
 'Lương: 20 Tr - 40 Tr VND' 'Lương: 10 Tr - 15 Tr VND'
 'Lương: 28 Tr - 35 Tr VND' 'Lương: 18 Tr - 23 Tr VND'
 'Lương: 11 Tr - 14 Tr VND' 'Lương: 2,500 - 4,300 USD'
 'Lương: 10 Tr - 19 Tr VND' 'Lương: 18 Tr - 20 Tr VND'
 'Lương: Trên 10 Tr VND' 'Lương: 45 Tr - 50 Tr VND'
 'Lương: 25 Tr - 30 Tr VND' 'Lương: 12 Tr - 16 Tr VND'
 'Lương: 15 Tr - 27 Tr VND' 'Lương: 15 Tr - 30 Tr VND'
 'Lương: 13 Tr - 17 Tr VND' 'Lương: 10,000 - 20,000 USD'
 'Lương: 35 Tr - 45 Tr VND' 'Lương: 15 Tr - 22 Tr VND'
 'Lương: 18 Tr - 22 Tr VND' 'Lương: 50 Tr - 80 Tr VND'
 'Lương: 25 Tr - 40 Tr VND' 'Lương: 1,000 - 1,500 USD'
 'Lươ

In [16]:

# Xóa dấu chấm cuối câu, xóa khoảng trống thừa, chuyển về chữ thường
df['salary'] = df['salary'].apply(lambda x: x.strip().lower())
# Delete Lương: in each cell
df['salary'] = df['salary'].apply(lambda x: re.sub(r'lương:', '', x).strip())
print(df['salary'].unique())

['cạnh tranh' '1,200 - 1,500 usd' '22 tr - 45 tr vnd' '10 tr - 13 tr vnd'
 '15 tr - 25 tr vnd' '8 tr - 16 tr vnd' 'trên 20 tr vnd'
 '8 tr - 20 tr vnd' '15 tr - 50 tr vnd' '500 - 1,000 usd'
 '10 tr - 18 tr vnd' '12 tr - 20 tr vnd' '15 tr - 20 tr vnd'
 '30 tr - 35 tr vnd' '20 tr - 40 tr vnd' '10 tr - 15 tr vnd'
 '28 tr - 35 tr vnd' '18 tr - 23 tr vnd' '11 tr - 14 tr vnd'
 '2,500 - 4,300 usd' '10 tr - 19 tr vnd' '18 tr - 20 tr vnd'
 'trên 10 tr vnd' '45 tr - 50 tr vnd' '25 tr - 30 tr vnd'
 '12 tr - 16 tr vnd' '15 tr - 27 tr vnd' '15 tr - 30 tr vnd'
 '13 tr - 17 tr vnd' '10,000 - 20,000 usd' '35 tr - 45 tr vnd'
 '15 tr - 22 tr vnd' '18 tr - 22 tr vnd' '50 tr - 80 tr vnd'
 '25 tr - 40 tr vnd' '1,000 - 1,500 usd' '7 tr - 15 tr vnd'
 '45 tr - 60 tr vnd' '8 tr - 12 tr vnd' '14 tr - 20 tr vnd'
 '18 tr - 24 tr vnd' '10 tr - 12,5 tr vnd' '8 tr - 10 tr vnd'
 '6 tr - 9 tr vnd' '6 tr - 11 tr vnd' '6 tr - 8 tr vnd'
 '20 tr - 35 tr vnd' '8 tr - 30 tr vnd' 'trên 18 tr vnd' '2 tr - 3 tr vnd'
 '7 tr - 18

In [24]:
# in raw_salary, delete all after  Số lượng tuyển 
df['salary'] = df['salary'].apply(lambda x: x.split('Số lượng tuyển')[0] if 'Số lượng tuyển' in x else x)
# in raw_salary, delete all before Thu nhập
df['salary'] = df['salary'].apply(lambda x: x.split('Thu nhập')[1] if 'Thu nhập' in x else x)
# delete blank
df['salary'] = df['salary'].apply(lambda x: x.strip())

In [ ]:
# Hàm xử lý dữ liệu lương
def process_salary(text):
    # Loại bỏ khoảng trắng thừa
    text = text.strip()
    
    # Loại bỏ đơn vị tiền tệ
    is_usd = 'usd' in text

    # Tách theo khoảng (vd: '10 tr - 15 tr vnd' hoặc '1,000 - 1,500 usd')
    range_match = re.match(r'([\d,\.]+)\s*-\s*([\d,\.]+)', text)
    if range_match:
        min_val = float(range_match.group(1).replace(',', '').replace('.', ''))
        max_val = float(range_match.group(2).replace(',', '').replace('.', ''))
        if 'tr' in text:
            min_val *= 1000000
            max_val *= 1000000
        elif is_usd:
            min_val *= usd_to_vnd  # Sử dụng biến tỷ giá USD đã có
            max_val *= usd_to_vnd
        return min_val, max_val

    # Trường hợp 'trên xx tr vnd'
    above_match = re.match(r'trên\s+([\d,\.]+)', text)
    if above_match:
        val = float(above_match.group(1).replace(',', '').replace('.', ''))
        if 'tr' in text:
            val *= 1000000
        elif is_usd:
            val *= usd_to_vnd
        return val, np.nan

    # Trường hợp 'lên đến xx tr vnd'
    below_match = re.match(r'lên đến\s+([\d,\.]+)', text)
    if below_match:
        val = float(below_match.group(1).replace(',', '').replace('.', ''))
        if 'tr' in text:
            val *= 1000000
        elif is_usd:
            val *= usd_to_vnd
        return np.nan, val

    return np.nan, np.nan

# Áp dụng hàm vào cột 'salary'
df[['min_salary', 'max_salary']] = df['salary'].apply(
    lambda x: pd.Series(process_salary(x))
)

# Hiển thị kết quả
df[['salary', 'min_salary', 'max_salary']].head()

,salary,min_salary,max_salary
0,trên 18 tr vnd,18000000.0,NaN
1,2 tr - 3 tr vnd,NaN,NaN
2,"1,000 - 1,500 usd",25485000.0,38227500.0


In [26]:
# in type của min_salary và max_salary
print(df['min_salary'].unique())

[18000000.       nan 25485000.]


In [35]:
# if max_salary > 100000000, print min_salary, max_salary, raw_salary
df.loc[df['max_salary'] > 100000000, ['min_salary', 'max_salary', 'raw_salary']]


,min_salary,max_salary,raw_salary
431,17839500.0,127425000.0,"700 - 5,000 USD"
1206,50970000.0,101940000.0,"2,000 - 4,000 USD"
1919,10000000.0,115000000.0,10 - 11.5 triệu
1923,11000000.0,135000000.0,11 - 13.5 triệu
2010,63712500.0,101940000.0,"2,500 - 4,000 USD"
2313,50970000.0,101940000.0,"2,000 - 4,000 USD"
2671,152910000.0,254850000.0,"6,000 - 10,000 USD"
3161,76455000.0,127425000.0,"3,000 - 5,000 USD"
3427,184000000.0,276000000.0,18.4 - 27.6 triệu
3601,89197500.0,165652500.0,"3,500 - 6,500 USD"


In [36]:
# print unique salary
print(df['raw_experience'].unique())

['1 năm' '3 năm' '2 năm' 'Không yêu cầu kinh nghiệm'
 '1 năm Hạn nộp hồ sơ: 20/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 'Dưới 1 năm' '5 năm' '4 năm' 'Trên 5 năm'
 '3 năm Hạn nộp hồ sơ: 25/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 'Dưới 1 năm Hạn nộp hồ sơ: 25/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 '3 năm Hạn nộp hồ sơ: 16/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 '1 nă'
 '1 năm Hạn nộp hồ sơ: 24/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 '1 năm Hạn nộp hồ sơ: 03/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 '2 năm Hạn nộp hồ sơ: 03/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 '3 năm Hạn nộp hồ sơ: 23/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 '1 năm Hạn nộp hồ sơ: 19/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 '1 năm Hạn nộp hồ sơ: 30/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 '5 nă'
 '1 năm Hạn nộp hồ sơ: 08/01/2025 Ứng tuyển ngay Lưu tin Ứng tuyển lại Nhắn ti'
 '2

In [37]:
# tính average_salary, (min + max) / 2
df['average_salary'] = (df['min_salary'] + df['max_salary']) / 2



In [38]:
# sort average_salary from min to max
df = df.sort_values(by='average_salary')
# without NaN
df = df.dropna(subset=['average_salary'])

df.tail(10)


,Unnamed: 0,jobtitle,jobdetail,jobname,companyscale,company_field,company_name,skill_offer1,skill_offer2,skill_offer3,skill_offer4,Location,general_info,raw_salary,raw_experience,position,work_type,min_salary,max_salary,average_salary
2010,https://www.topcv.vn/viec-lam/fullstack-java/1...,Fullstack\n \n Java,Fullstack\n \n Java\n ...,Fullstack Developer,25-99,It - Phần Mềm,Công Ty Tnhh Giải Pháp Phân Tích Dữ Liệu Insig...,Git,Rest Api,Kiến Trúc Phần Mềm,"Sql Và Cơ Sở Dữ Liệu Quan Hệ (mysql, Postgresq...",Hồ Chí Minh,Thông tin chung\n \n \n \n ...,"2,500 - 4,000 USD",Trên 5 năm,Nhân viên,Toàn thời gian,63712500.0,101940000.0,82826250.0
3322,https://www.topcv.vn/viec-lam/giam-doc-trung-t...,Giám Đốc\n \n Trung\n ...,Giám Đốc\n \n Trung\n ...,IT Project Manager,500-1000,It - Phần Mềm,Công Ty Cổ Phần Tập Đoàn Meey Land,Phân Tích Dữ Liệu,Quản Lý Nhân Sự,Quản Lý Dự Án,Lập Kế Hoạch Chiến Lược,Hà Nội,Thông tin chung\n \n \n \n ...,70 - 100 triệu,Trên 5 năm,Giám đốc,Toàn thời gian,70000000.0,100000000.0,85000000.0
3811,https://www.topcv.vn/brand/smartosc/tuyen-dung...,NaN,Embedded Software Manager (3000$ - 4000$)\n ...,Embedded Engineer/Lập trình nhúng,1000 - 2999,Khác,Công Ty Cổ Phần Smartosc,Quản Lý Dự Án Phần Mềm Nhúng,Kiến Thức Sâu Về Kiến Trúc Phần Mềm Nhúng,Kinh Nghiệm Với Các Vi Điều Khiển Và Vi Xử Lý,"Địa Điểm Làm Việc - Hà Nội: Tòa Handico, Phạm ...",,,"3,000 - 4,000 USD",Trên 5 năm Ứng tuyển ngay Lưu tin Ứng tuyển lạ...,Nhân viên,,76455000.0,101940000.0,89197500.0
3161,https://www.topcv.vn/viec-lam/ai-explorer-engi...,AI Explorer\n Engineer,AI Explorer\n Engineer\n \n ...,AI Engineer,25-99,It - Phần Mềm,Công Ty Tnhh Công Nghệ Blue Belt,Machine Learning,Large Language Model,Deep Learning,Tensorflow/pytorch,Hà Nội,Thông tin chung\n \n \n \n ...,"3,000 - 5,000 USD",3 năm,Nhân viên,Toàn thời gian,76455000.0,127425000.0,101940000.0
3652,https://www.topcv.vn/viec-lam/middle-web-syste...,Middle Web\n System Engineer\n ...,Middle Web\n System Engineer\n ...,Software Engineer,25-99,It - Phần Mềm,Công Ty Tnhh Otani U.p.,Python,Vuejs,Chủ Động Trong Công Việc,,Hà Nội,Thông tin chung\n \n \n \n ...,18.5 - 25 triệu,2 năm,Nhân viên,Toàn thời gian,185000000.0,25000000.0,105000000.0
3601,https://www.topcv.vn/viec-lam/it-delivery-mana...,IT\n Delivery Manager Tại\n ...,IT\n Delivery Manager Tại\n ...,Technical Manager,100-499,It - Phần Mềm,Công Ty Cp Phần Mềm Mor,Project Management,Communication,People Management,Japanese,,Thông tin chung\n \n \n \n ...,"3,500 - 6,500 USD",Trên 5 năm,Trưởng/Phó phòng,Toàn thời gian,89197500.0,165652500.0,127425000.0
4164,https://www.topcv.vn/viec-lam/user-experience-...,User Experience Assurance-User Oriented Testers,User Experience Assurance-User Oriented Tester...,Software Tester (Automation & Manual),25-99,It - Phần Mềm,Công Ty Tnhh Otani U.p.,Tester,User Research,,,Hà Nội,Thông tin chung\n \n \n \n ...,11.5 - 18.5 triệu,2 năm,Nhân viên,Toàn thời gian,115000000.0,185000000.0,150000000.0
4146,https://www.topcv.vn/viec-lam/ky-su-kiem-thu-p...,Kỹ Sư\n \n Kiểm Thử Phần Mềm...,Kỹ Sư\n \n Kiểm Thử Phần Mềm...,Manual Tester,100-499,It - Phần Mềm,Công Ty Cổ Phần Its Global,Tester,Tiếng Nhật N2 - N1,Api,,Hà Nội,Thông tin chung\n \n \n \n ...,150 - 200 triệu,2 năm,Nhân viên,Toàn thời gian,150000000.0,200000000.0,175000000.0
2671,https://www.topcv.vn/viec-lam/giam-doc-sale-th...,Giám Đốc\n \n Sale Thị Trườn...,Giám Đốc\n \n Sale Thị Trườn...,Sales Director/Giám đốc kinh doanh,100-499,It - Phần Mềm,Công Ty Tnhh Phát Triển Hệ Thống Dịch Vụ Công ...,Kỹ Năng Đàm Phán Và Thương Lượng,"Tiếng Nhật (nói, Đọc, Viết)",Kinh Nghiệm Quản Lý Đội Nhóm,Kinh Nghiệm Bán Hàng Quốc Tế,,Thông tin chung\n \n \n \n ...,"6,000 - 10,000 USD",Trên 5 năm,Giám đốc,Toàn thời gian,152910000.0,254850000.0,203880000.0
3427,https://www.topcv.vn/viec-lam/ky-su-phat-trien...,Kỹ Sư\n \n Phát Triển Ứng Dụ...,Kỹ Sư\n \n Phát Triển Ứng Dụ...,Fullstack Developer,100-499,It - Phần Mềm,Cty Tnhh R Techno Việt Nam,Phân Tích Yêu Cầu,Phát Triển Ứng Dụng,Kiểm Thử 

In [39]:
# print 10 dòng đầu tiên của 3 cột mới
print(df[['min_salary', 'max_salary', 'average_salary']].head(10))

      min_salary  max_salary  average_salary
4314         0.0         0.0             0.0
319          0.0         0.0             0.0
332          0.0         0.0             0.0
1233         0.0         0.0             0.0
3775         0.0         0.0             0.0
4270         0.0         0.0             0.0
3808         0.0         0.0             0.0
2867         0.0         0.0             0.0
3643         0.0         0.0             0.0
884          0.0         0.0             0.0


In [40]:
# if find "Trên" or "Tới" in raw_salary, set min_salary = max_salary = average_salary = this number

def process_salary2(text):
    # Loại bỏ khoảng trắng thừa
    text = text.strip()
    
    # Nếu không tìm thấy 'Trên' hoặc 'Tới' thì trả về NaN
    if 'Trên' not in text and 'Tới' not in text:
        return np.nan, np.nan, np.nan
    
    # Loại bỏ các ký tự không phải là số
    number = re.sub(r'\D', '', text)
    
    # Kiểm tra nếu number rỗng
    if not number:
        return np.nan, np.nan, np.nan
    
    # Chuyển đổi thành số nguyên
    number = int(number)
    
    # Nếu tìm thấy 'triệu' hoặc 'tr' thì nhân với 1000000
    if any(unit in text.lower() for unit in ['triệu', 'tr']):
        number *= 1000000
    
    # Nếu tìm thấy 'USD' thì nhân với 25485
    elif 'USD' in text.upper():
        number *= 25485
    
    return number, number, number

# Áp dụng hàm cho các hàng đang bị NaN. nếu không thì bỏ qua
df.loc[df['min_salary'].isnull(), ['min_salary', 'max_salary', 'average_salary']] = df.loc[df['min_salary'].isnull(), 'raw_salary'].apply(process_salary2)


In [41]:
# xóa dấu cách thừa trong raw_salary
df['raw_salary'] = df['raw_salary'].apply(lambda x: re.sub(r'\s+', ' ', x))
# if min_salary = Nan, print unique raw_salary
print(df[df['min_salary'].isnull()]['raw_salary'].unique())



[]


In [42]:
# print min_salary, max_salary, average_salary
print(df[['min_salary', 'max_salary', 'average_salary']].head(100))


      min_salary  max_salary  average_salary
4314         0.0         0.0             0.0
319          0.0         0.0             0.0
332          0.0         0.0             0.0
1233         0.0         0.0             0.0
3775         0.0         0.0             0.0
...          ...         ...             ...
789    1000000.0   7000000.0       4000000.0
1729   3000000.0   6000000.0       4500000.0
1161   3000000.0   6000000.0       4500000.0
1730   3000000.0   6000000.0       4500000.0
1181   2000000.0   7000000.0       4500000.0

[100 rows x 3 columns]


In [43]:
#print null min_salary
print(df['min_salary'].isnull().sum())

0


## Experience

In [44]:
# Delete all after Hạn nộp hồ sơ
df['raw_experience'] = df['raw_experience'].apply(lambda x: x[:x.find('Hạn nộp hồ sơ')])

# Xóa dấu chấm cuối câu, xóa khoảng trống thừa, chuyển về chữ thường raw_experience
df['raw_experience'] = df['raw_experience'].apply(lambda x: x.strip().lower())


In [45]:
# Hàm xử lý dữ liệu kinh nghiệm
# lấy số duy nhất xuất hiện, nếu thấy "không" thì lưu là 0

def process_experience(text):
    # Loại bỏ khoảng trắng thừa
    text = text.strip()
    
    #delete "nă"
    text = re.sub(r'nă', '', text)
    
    # Nếu không tìm thấy 'không' thì trả về NaN
    if 'không' in text:
        return 0
    
    # Tìm số trong văn bản
    number = re.findall(r'\d+', text)
    
    # Kiểm tra nếu number rỗng
    if not number:
        return np.nan
    
    # Chuyển đổi thành số nguyên
    number = int(number[0])
    
    return number

# Áp dụng hàm cho toàn bộ cột 'raw_experience'
df['experience'] = df['raw_experience'].apply(process_experience)

# In kết quả
print(df[['raw_experience', 'experience']].head(10))


     raw_experience  experience
4314           4 nă           4
319            4 nă           4
332            4 nă           4
1233           1 nă           1
3775           4 nă           4
4270           3 nă           3
3808           5 nă           5
2867      trên 5 nă           5
3643           4 nă           4
884            3 nă           3


In [46]:
#print null experience
print(df['experience'].isnull().sum())

0


# PREPROCESS - MISSING VALUES

### Kiểm tra dữ liệu sau khi thêm cột


In [47]:
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 2233 entries, 4314 to 3427
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      2233 non-null   object 
 1   jobtitle        2094 non-null   object 
 2   jobdetail       2233 non-null   object 
 3   jobname         2233 non-null   object 
 4   companyscale    2233 non-null   object 
 5   company_field   2233 non-null   object 
 6   company_name    2233 non-null   object 
 7   skill_offer1    2233 non-null   object 
 8   skill_offer2    2233 non-null   object 
 9   skill_offer3    2233 non-null   object 
 10  skill_offer4    2233 non-null   object 
 11  Location        2233 non-null   object 
 12  general_info    2233 non-null   object 
 13  raw_salary      2233 non-null   object 
 14  raw_experience  2233 non-null   object 
 15  position        2233 non-null   object 
 16  work_type       2233 non-null   object 
 17  min_salary      2233 non-null   flo

In [48]:
# Xóa các cột không cần thiết
df = df.drop(columns=['jobdetail', 'jobtitle', 'general_info', 'raw_salary', 'raw_experience', 'Unnamed: 0'])

In [49]:
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 2233 entries, 4314 to 3427
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   jobname         2233 non-null   object 
 1   companyscale    2233 non-null   object 
 2   company_field   2233 non-null   object 
 3   company_name    2233 non-null   object 
 4   skill_offer1    2233 non-null   object 
 5   skill_offer2    2233 non-null   object 
 6   skill_offer3    2233 non-null   object 
 7   skill_offer4    2233 non-null   object 
 8   Location        2233 non-null   object 
 9   position        2233 non-null   object 
 10  work_type       2233 non-null   object 
 11  min_salary      2233 non-null   float64
 12  max_salary      2233 non-null   float64
 13  average_salary  2233 non-null   float64
 14  experience      2233 non-null   int64  
dtypes: float64(3), int64(1), object(11)
memory usage: 279.1+ KB
None
         min_salary    max_salary  average_salary   experien

In [50]:
#Với mỗi cột, đếm xem có bao nhiêu giá trị ''
print((df == '').sum())


jobname             0
companyscale        7
company_field       0
company_name        0
skill_offer1      539
skill_offer2      616
skill_offer3      729
skill_offer4      865
Location           51
position            0
work_type          19
min_salary          0
max_salary          0
average_salary      0
experience          0
dtype: int64


In [51]:
#delete all row have '' in company_scale, Location, work_type
df = df[df['companyscale'] != '']
df = df[df['Location'] != '']
df = df[df['work_type'] != '']
#detêt all row if min_salary = null
df = df[df['min_salary'].notnull()]

In [52]:
#print last 5 rows
print(df.tail())

                                    jobname companyscale  company_field  \
3161                            AI Engineer        25-99  It - Phần Mềm   
3652                      Software Engineer        25-99  It - Phần Mềm   
4164  Software Tester (Automation & Manual)        25-99  It - Phần Mềm   
4146                          Manual Tester      100-499  It - Phần Mềm   
3427                    Fullstack Developer      100-499  It - Phần Mềm   

                          company_name       skill_offer1  \
3161  Công Ty Tnhh Công Nghệ Blue Belt   Machine Learning   
3652           Công Ty Tnhh Otani U.p.             Python   
4164           Công Ty Tnhh Otani U.p.             Tester   
4146        Công Ty Cổ Phần Its Global             Tester   
3427        Cty Tnhh R Techno Việt Nam  Phân Tích Yêu Cầu   

              skill_offer2              skill_offer3        skill_offer4  \
3161  Large Language Model             Deep Learning  Tensorflow/pytorch   
3652                 Vuejs  Ch

In [53]:
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 2175 entries, 4314 to 3427
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   jobname         2175 non-null   object 
 1   companyscale    2175 non-null   object 
 2   company_field   2175 non-null   object 
 3   company_name    2175 non-null   object 
 4   skill_offer1    2175 non-null   object 
 5   skill_offer2    2175 non-null   object 
 6   skill_offer3    2175 non-null   object 
 7   skill_offer4    2175 non-null   object 
 8   Location        2175 non-null   object 
 9   position        2175 non-null   object 
 10  work_type       2175 non-null   object 
 11  min_salary      2175 non-null   float64
 12  max_salary      2175 non-null   float64
 13  average_salary  2175 non-null   float64
 14  experience      2175 non-null   int64  
dtypes: float64(3), int64(1), object(11)
memory usage: 271.9+ KB
None
         min_salary    max_salary  average_salary   experien

# PREPROCESS - TRANSLATE TO ENGLISH

In [54]:
%pip install googletrans==4.0.0-rc1


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [55]:
import pandas as pd
from googletrans import Translator
from tqdm import tqdm

# Các cột cần dịch
columns_to_translate = [
    "jobname",
    "company_field",
    "company_name",
    "skill_offer1",
    "skill_offer2",
    "skill_offer3",
    "skill_offer4",
    "position",
    "work_type"
]

# Khởi tạo đối tượng Translator
translator = Translator()

# Hàm dịch nội dung

def translate_text(text):
    try:
        if pd.notnull(text):  # Kiểm tra nếu ô không rỗng
            translation = translator.translate(text, src='auto', dest='en')
            return translation.text if translation and translation.text else text
        return text
    except Exception as e:
        print(text)
        return text

# Chỉ lấy 5 hàng đầu tiên để thử nghiệm
data_sample = df.head(5).copy()

# Dịch từng cột với tqdm để hiển thị tiến độ
for column in columns_to_translate:
    if column in data_sample.columns:
        print(f"Đang dịch cột: {column}")
        data_sample[column] = [translate_text(text) for text in tqdm(data_sample[column], desc=f"Dịch {column}")]

# Kiểm tra kết quả
print(data_sample)


Đang dịch cột: jobname


Dịch jobname: 100%|██████████| 5/5 [00:06<00:00,  1.25s/it]


Đang dịch cột: company_field


Dịch company_field: 100%|██████████| 5/5 [00:04<00:00,  1.08it/s]


Đang dịch cột: company_name


...

Đang dịch cột: position


Dịch position: 100%|██████████| 5/5 [00:05<00:00,  1.12s/it]


Đang dịch cột: work_type


Dịch work_type: 100%|██████████| 5/5 [00:05<00:00,  1.11s/it]

                                    jobname companyscale  company_field  \
4314                      Software business        25-99        Finance   
319                     Fullstack Developer     500-1000  IT - Software   
332                          Game Developer        25-99  IT - Software   
1233          Product Owner/Product Manager        25-99  IT - Software   
3775  Software Tester (Automation & Manual)        25-99  IT - Software   

                                   company_name        skill_offer1  \
4314                Paytech Joint Stock Company             Present   
319   CMC Technology & Solution Company Limited                       
332                              Gamify Studios                       
1233                             Gamify Studios               Agile   
3775         Live Group Vietnam Company Limited  Project management   

                skill_offer2      skill_offer3           skill_offer4  \
4314  Building relationships         Negotiate   

## Translate Jobname

In [56]:
# Delete all in ( ) in jobname
df['jobname'] = df['jobname'].apply(lambda x: re.sub(r'\(.*\)', '', x).strip())

In [57]:
# save jobname to temp array
jobname_array = df['jobname'].unique()
jobname_array

array(['Kinh doanh phần mềm', 'Fullstack Developer', 'Game Developer',
       'Product Owner/Product Manager', 'Software Tester', 'UI/UX Design',
       'System Architect', 'Business Analyst', 'Công nghệ thông tin khác',
       'Data Analyst', 'Thiết kế đồ họa', 'Mobile Developer',
       'Frontend Developer', 'Bán hàng kỹ thuật IT', 'IT Consultant',
       'Game Tester', 'Backend Developer', 'AI Engineer',
       'Software Engineer', 'Automation Tester', 'Kỹ sư cầu nối BrSE',
       'Sales IT Phần mềm khác', 'Database Administrator',
       'Brand Marketing', 'DevOps Engineer',
       'Kinh doanh Domain/Hosting/Server', 'System Engineer',
       'Manual Tester', 'Data Scientist', 'Illustration', 'Game Design',
       'Concept Artist', 'Embedded Engineer/Lập trình nhúng', 'Kỹ sư IoT',
       'IT Helpdesk/IT support', 'Data Labeling', 'Kỹ thuật IT',
       'Data Engineer', 'Cloud Engineer', 'Vị trí Game Development khác',
       'Animation Design', 'Quản trị và vận hành bảo mật',
      

In [58]:
# translate jobname_array
jobname_array = [translate_text(text) for text in tqdm(jobname_array, desc="Dịch jobname")]


Dịch jobname: 100%|██████████| 73/73 [00:48<00:00,  1.51it/s]


In [59]:
# create dictionary to save jobname_array
jobname_dict = dict(zip(df['jobname'].unique(), jobname_array))
# replace jobname by jobname_dict
df['jobname'] = df['jobname'].replace(jobname_dict)

In [60]:
# print unique jobname
print(df['jobname'].unique())

['Software business' 'Fullstack Developer' 'Game Developer'
 'Product Owner/Product Manager' 'Software tester' 'UI/UX Design'
 'System Architect' 'Business Analyst' 'Other information technology'
 'Data Analyst' 'Graphic design' 'Mobile Developer' 'Frontend Developer'
 'IT technical sales' 'IT Consultant' 'Game Tester' 'Backend Developer'
 'AI Engineer' 'Software Engineer' 'Automation Tester'
 'Bridge Bridge Bridge Bridge' 'Sales IT other software'
 'Database Administrator' 'Brand Marketing' 'DevOps Engineer'
 'Domain/hosting/server business' 'System Engineer' 'Manual Tester'
 'Data Scientist' 'Illustration' 'Game Design' 'Concept Artist'
 'Embedded Engineer/Dip programming' 'IoT engineer'
 'IT Helpdesk/IT support' 'Data Labeling' 'IT technique' 'Data Engineer'
 'Cloud Engineer' 'Other Development game position' 'Animation Design'
 'Management and operation of confidentiality' 'System Administrator'
 'QA Engineer' 'Other subject teachers' 'Computer teacher'
 'Network Engineer' 'Process

## Translate company_field

In [61]:
# save company_field to temp array
company_field_array = df['company_field'].unique()
company_field_array


array(['Tài Chính', 'It - Phần Mềm', 'Viễn Thông', 'Ngân Hàng',
       'Bán Lẻ - Hàng Tiêu Dùng - Fmcg', 'Bất Động Sản',
       'Internet / Online', 'Khác', 'Tư Vấn', 'Xuất Nhập Khẩu',
       'Xây Dựng', 'Điện Tử / Điện Lạnh',
       'Marketing / Truyền Thông / Quảng Cáo', 'Du Lịch',
       'Giáo Dục / Đào Tạo', 'Thương Mại Điện Tử',
       'Dược Phẩm / Y Tế / Công Nghệ Sinh Học',
       'Agency (marketing/advertising)', 'Tự Động Hóa', 'Sản Xuất',
       'Giải Trí', 'Nhân Sự', 'Nhà Hàng / Khách Sạn', 'In Ấn / Xuất Bản',
       'It - Phần Cứng', 'Logistics - Vận Tải', 'Thời Trang',
       'Chứng Khoán', 'Thiết Kế / Kiến Trúc', 'Kế Toán / Kiểm Toán',
       'Nông Lâm Ngư Nghiệp', 'Bảo Trì / Sửa Chữa', 'Bảo Hiểm', 'Cơ Khí',
       'Năng Lượng'], dtype=object)

In [62]:
# translate company_field_array
company_field_array = [translate_text(text) for text in tqdm(company_field_array, desc="Dịch company_field")]

# create dictionary to save company_field_array
company_field_dict = dict(zip(df['company_field'].unique(), company_field_array))
# replace company_field by company_field_dict
df['company_field'] = df['company_field'].replace(company_field_dict)

Dịch company_field: 100%|██████████| 35/35 [00:27<00:00,  1.27it/s]


In [63]:
df['company_field'].unique()

array(['Finance', 'IT - Software', 'Telecommunications', 'Bank',
       'Retail - Consumer goods - FMCG', 'Real estate',
       'Internet / Online', 'Other', 'Advise', 'Import and export',
       'Build', 'Electronics / Refrigeration',
       'Marketing / Communications / Advertising', 'Tourism',
       'Education / Training', 'E -commerce',
       'Pharmaceutical / Health / Biotechnology',
       'Agency (marketing/advertising)', 'Automation', 'Manufacture',
       'Entertainment', 'Personnel', 'Restaurant / hotel',
       'Print / publish', 'IT - Hardware', 'Logistics - Transport',
       'Fashion', 'Stock', 'Design / Architecture',
       'Accounting / Auditing', 'Agriculture and forestry',
       'Maintenance / Repair', 'Insurance', 'Mechanical', 'Energy'],
      dtype=object)

## Translate position

In [64]:
# save position to temp array
position_array = df['position'].unique()
position_array

array(['Trưởng/Phó phòng', 'Nhân viên', 'Quản lý / Giám sát',
       'Thực tập sinh', 'Trưởng nhóm', 'Trưởng chi nhánh', 'Giám đốc',
       'Phó giám đốc'], dtype=object)

In [65]:
#translate position_array
position_array = [translate_text(text) for text in tqdm(position_array, desc="Dịch position")]

# create dictionary to save position_array
position_dict = dict(zip(df['position'].unique(), position_array))
# replace position by position_dict
df['position'] = df['position'].replace(position_dict)


Dịch position: 100%|██████████| 8/8 [00:05<00:00,  1.36it/s]


In [66]:
df.head(10)

,jobname,companyscale,company_field,company_name,skill_offer1,skill_offer2,skill_offer3,skill_offer4,Location,position,work_type,min_salary,max_salary,average_salary,experience
4314,Software business,25-99,Finance,Công Ty Cổ Phần Paytech,Thuyết Trình,Xây Dựng Mối Quan Hệ,Đàm Phán,Quản Lý Dự Án,Hà Nội,Head/Deputy Head,Toàn thời gian,0.0,0.0,0.0,4
319,Fullstack Developer,500-1000,IT - Software,Công Ty Tnhh Tổng Công Ty Công Nghệ & Giải Phá...,,,,,Hà Nội,Staff,Toàn thời gian,0.0,0.0,0.0,4
332,Game Developer,25-99,IT - Software,Gamify Studios,,,,,Hồ Chí Minh,Staff,Toàn thời gian,0.0,0.0,0.0,4
1233,Product Owner/Product Manager,25-99,IT - Software,Gamify Studios,Agile,Scrum,Product Strategy,Product Roadmap,Hồ Chí Minh,Staff,Toàn thời gian,0.0,0.0,0.0,1
3775,Software tester,25-99,IT - Software,Công Ty Tnhh Live Group Việt Nam,Quản Lý Dự Án,Kiểm Thử Phần Mềm,Lập Kế Hoạch Kiểm Thử,Thiết Kế Test Case,Hồ Chí Minh,Management / supervision,Toàn thời gian,0.0,0.0,0.0,4
4270,UI/UX Design,25-99,IT - Software,Gamify Studios,Thiết Kế Giao Diện Người Dùng (ui),Thiết Kế Trải Nghiệm Người Dùng (ux),Nghiên Cứu Người Dùng,"Sử Dụng Phần Mềm Thiết Kế (figma, Adobe Xd, Sk...",Hồ Chí Minh,Staff,Toàn thời gian,0.0,0.0,0.0,3
3808,System Architect,5000 - 9999,Telecommunications,Công Ty Cổ Phần Viễn Thông Fpt,Quản Lý Dự Án,Thiết Kế Hệ Thống,Lập Trình Hướng Đối Tượng,Phát Triển Phần Mềm,Hồ Chí Minh,Staff,Toàn thời gian,0.0,0.0,0.0,5
2867,Business Analyst,25-99,IT - Software,Công Ty Tnhh Công Nghệ Blue Belt,Kỹ Năng Lãnh Đạo,Quản Lý Dự Án,Phân Tích Nghiệp Vụ,Giao Tiếp Hiệu Quả,Hà Nội,Head/Deputy Head,Toàn thời gian,0.0,0.0,0.0,5
3643,Fullstack Developer,100-499,IT - Software,Opencommerce Group,Python,Vuejs,Product Development,Sql/nosql,Hà Nội,Staff,Toàn thời gian,0.0,0.0,0.0,4
884,Other information technology,5000 - 9999,Bank,Ngân Hàng Tmcp An Bình Pro Company,,,,,Hà Nội,Staff,Toàn thời gian,0.0,0.0,0.0,3


## Translate work_type

In [67]:
# save work_type to temp array
work_type_array = df['work_type'].unique()
work_type_array


array(['Toàn thời gian', 'Thực tập', 'Bán thời gian'], dtype=object)

In [68]:
#translate work_type_array
work_type_array = [translate_text(text) for text in tqdm(work_type_array, desc="Dịch work_type")]

# create dictionary to save work_type_array
work_type_dict = dict(zip(df['work_type'].unique(), work_type_array))
# replace work_type by work_type_dict
df['work_type'] = df['work_type'].replace(work_type_dict)


Dịch work_type: 100%|██████████| 3/3 [00:02<00:00,  1.21it/s]


## Skill offer

In [ ]:
import json

# Đọc file JSON mới được tải lên
json_path = 'final_skill_groups.json'
with open(json_path, 'r', encoding='utf-8') as f:
    skill_groups = json.load(f)

# Tạo cột mới trong DataFrame để chứa nhóm kỹ năng
def classify_skill(skill):
    if pd.isna(skill) or skill == '':
        return ''
    for group, skills in skill_groups.items():
        if skill in skills:
            return group
    return 'Other'

# Áp dụng phân loại cho từng cột kỹ năng
for col in df.columns:
    df[f'{col}_group'] = df[col].apply(lambda x: classify_skill(x))

# Loại bỏ giá trị trùng lặp trong cùng một hàng
group_cols = [col for col in df.columns if '_group' in col]

# Xử lý từng hàng để loại bỏ giá trị trùng lặp
for index, row in df.iterrows():
    seen = set()
    for col in group_cols:
        if row[col] in seen:
            df.at[index, col] = ''  # Xóa giá trị trùng lặp
        else:
            seen.add(row[col])
 
# Xuất file CSV sau khi ghép nhóm kỹ năng
output_csv_path = 'cleanded_database.csv'
df.to_csv(output_csv_path, index=False)

output_csv_path

'cleanded_database.csv'